In [1]:
!pip install trackpy
import trackpy


C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage
from scipy.stats import iqr
from ipywidgets import interact
import os
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.restoration import rolling_ball
from skimage.measure import label, regionprops
from pandas import DataFrame as df
sns.set_style('dark', rc={'image.cmap':'inferno'})

In [3]:
image_names=os.listdir('../Data/41/')
fluorescent_image_names=[el for el in image_names if 'w3RFPled' in el]
ordering=np.argsort([int(el.split('_t')[1][:-4]) for el in fluorescent_image_names])
ordered_fluo_imgs = ['../Data/41/'+fluorescent_image_names[i] for i in ordering]
ordered_fluo_imgs

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../Data/41/'

In [ ]:
@interact
def preprocess(i=(0,100)):
    fig,ax=plt.subplots(1,2,figsize=(10,5))
    image = imread(ordered_fluo_imgs[int(i)])
    corrected_image = image - rolling_ball(image,radius=20)
    ax[0].imshow(image)
    
    #preprocess
    thresholded=corrected_image>threshold_otsu(corrected_image)
    
    ax[1].imshow(thresholded,cmap='Greys_r')
    ax[1].set_title('N_objects= '+str(len(regionprops(label(thresholded)))))

In [ ]:
def make_mask(image):
    corrected_image = image - rolling_ball(image,radius=20)
    thresholded=corrected_image>threshold_otsu(corrected_image)        
    return thresholded
frames=[make_mask(imread(image_name)) for image_name in ordered_fluo_imgs[:20]]

In [ ]:
f = trackpy.locate(frames[0], 15, invert=True) #this is terrible
trackpy.annotate(f,frames[0])

In [ ]:
f

In [ ]:
our_dataframe=df(columns=['y','x','mass','size','frame'])
for idx,thresholded_img in enumerate(frames):
    regions=regionprops(label(thresholded_img))
    for r in regions:
        x,y=r.centroid
        mass=r.area
        size=r.equivalent_diameter
        our_dataframe.loc[len(our_dataframe)]=x,y,mass,size,idx

In [ ]:
our_dataframe

In [ ]:
tracked=trackpy.link(our_dataframe,25,memory=2)

In [ ]:
tracked

In [ ]:
trajectories=[np.asarray(tracked[tracked['particle']==i][['x','y']]) for i in range(np.max(tracked['particle']+1))]

In [ ]:
from matplotlib.colors import hsv_to_rgb

colors = [hsv_to_rgb( (0.5+0.5*np.random.rand(3))) for i in range(len(trajectories))]

@interact
def plot_result(i=(0,20)):
    fig,ax=plt.subplots(1,1,figsize=(10,10))
    image = frames[i]
    ax.imshow(image)
    for traj_idx,traj in enumerate(trajectories):
        x,y=traj.T
        x=x[:i]
        y=y[:i]
        plt.plot(x,y,color='r')#colors[traj_idx])
  

In [4]:
colors

UsageError: %colors: you must specify a color scheme. See '%colors?'
